<a href="https://colab.research.google.com/github/srinath-96/SubRedditAgent/blob/main/AdvancedRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Make sure necessary libraries are installed
!pip install -q google-adk google-generativeai langchain langchain-google-genai chromadb jq pydantic_core tiktoken
# tiktoken is often needed by LangChain text splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:00:

In [18]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [4]:
import os
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types as adk_types
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import asyncio
import traceback

In [19]:
# LangChain Imports
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
# --- Imports for Parent Document Retriever ---
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore # Simple in-memory store for parent docs
# --- End Imports for Parent Document Retriever ---

In [6]:


try:


  if 'GOOGLE_API_KEY' not in os.environ:

      from google.colab import userdata
      os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

  if not os.environ.get('GOOGLE_API_KEY'):
      print("ERROR: GOOGLE_API_KEY not found.")
      print("Please set it using Colab Secrets (recommended) or by uncommenting and")
      print("replacing 'YOUR_API_KEY' in the code above.")

  else:
      genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
      print("Google API Key configured successfully.")

except ImportError:
    print("Could not import google.colab.userdata. Are you running in Colab?")
    if not os.environ.get('GOOGLE_API_KEY'):
        print("ERROR: GOOGLE_API_KEY not found. Please set the environment variable.")

except Exception as e:
    print(f"An error occurred during API key configuration: {e}")


Google API Key configured successfully.


In [7]:
import json
from google.colab import files

print("Please upload your Reddit posts JSON file:")
uploaded = files.upload()

json_filename = None
if uploaded:
    json_filename = list(uploaded.keys())[0]
    print(f"\nUploaded file: '{json_filename}'")
else:
    print("No file uploaded. Cannot proceed with data loading.")


Please upload your Reddit posts JSON file:


Saving wallstreetbets_week_250posts_20250425_183349.json to wallstreetbets_week_250posts_20250425_183349.json

Uploaded file: 'wallstreetbets_week_250posts_20250425_183349.json'


In [21]:
# Global variables for retriever components
vectorstore = None
parent_store = None
retriever = None
embeddings = None # Define embeddings globally as well

# Proceed only if the JSON file was uploaded and API key is set
if json_filename and os.environ.get('GOOGLE_API_KEY'):
    print(f"\n--- Setting up LangChain Parent Document Retriever for {json_filename} ---")

    try:
        # 1. Define Metadata Extraction Function
        def metadata_func(record: dict, metadata: dict) -> dict:
            metadata["post_id"] = record.get("id", "N/A")
            metadata["url"] = record.get("url", "N/A")
            metadata["score"] = record.get("score", 0)
            # Ensure source matches the filename for the loader
            metadata["source"] = json_filename # Add source filename to metadata
            return metadata

        # 2. Initialize JSONLoader
        loader = JSONLoader(
            file_path=json_filename,
            jq_schema='.[ ] | {content: ("Title: " + .title + "\nBody: " + (.body // "")), id: .id, url: .url, score: .score}',
            content_key="content",
            metadata_func=metadata_func,
            text_content=False
        )

        # 3. Load Documents
        print("Loading documents via LangChain JSONLoader...")
        documents = loader.load()
        # Filter out any potentially empty documents resulting from jq schema/source data
        documents = [doc for doc in documents if doc.page_content.strip() and doc.page_content != "Title: \nBody:"]
        print(f"Loaded and filtered {len(documents)} documents.")

        if documents:
            # 4. Initialize Stores
            print("Initializing parent docstore (InMemoryStore)...")
            parent_store = InMemoryStore() # Store for large parent chunks

            print("Initializing Chroma vectorstore for child documents...")
            # Ensure embedding function is initialized
            embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # Use appropriate model
            vectorstore = Chroma(
                collection_name="reddit_child_chunks_v2", # Changed name slightly
                embedding_function=embeddings
                # persist_directory="./chroma_db_reddit_pdr" # Optional: persist to disk
            )

            # 5. Initialize Splitters
            # Larger chunks for parent documents (the context returned to LLM)
            parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
            # Smaller chunks for child documents (used for embedding and search)
            child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
            print("Initialized parent and child text splitters.")

            # 6. Initialize ParentDocumentRetriever
            print("Initializing ParentDocumentRetriever...")
            retriever = ParentDocumentRetriever(
                vectorstore=vectorstore,    # Store for child vectors
                docstore=parent_store,      # Store for parent documents
                child_splitter=child_splitter,
                parent_splitter=parent_splitter # Uses child_splitter if None, explicitly providing parent_splitter here
            )

            # 7. Add Documents to the Retriever (Using ids=None)
            print(f"Adding {len(documents)} documents to the retriever (this may take time)...")

            # --- MODIFIED LINE: Let the retriever generate IDs ---
            retriever.add_documents(documents, ids=None, add_to_docstore=True)
            # --- END MODIFICATION ---

            # (The lines creating 'doc_ids' manually are no longer needed and can be removed/commented)
            # # Optional: Provide unique IDs if available and desired (REMOVED)
            # # doc_ids = [doc.metadata.get("post_id", f"doc_{i}") for i, doc in enumerate(documents)] # REMOVED

            print("Documents added successfully to ParentDocumentRetriever.")
            print(f"Parent store size: {len(list(parent_store.yield_keys()))} keys added.")
            # You can try querying the vectorstore count if needed, e.g., vectorstore._collection.count() if using Chroma directly

        else:
            print("No documents loaded, skipping retriever setup.")

    except Exception as e:
        print(f"An error occurred during Parent Document Retriever setup: {e}")
        traceback.print_exc()
        # Reset global vars on error to prevent issues later
        vectorstore = None
        parent_store = None
        retriever = None
        embeddings = None
else:
    if not json_filename:
        print("Skipping LangChain setup: JSON file not uploaded.")
    else:
        print("Skipping LangChain setup: Google API Key not configured.")


--- Setting up LangChain Parent Document Retriever for wallstreetbets_week_250posts_20250425_183349.json ---
Loading documents via LangChain JSONLoader...
Loaded and filtered 250 documents.
Initializing parent docstore (InMemoryStore)...
Initializing Chroma vectorstore for child documents...
Initialized parent and child text splitters.
Initializing ParentDocumentRetriever...
Adding 250 documents to the retriever (this may take time)...
Documents added successfully to ParentDocumentRetriever.
Parent store size: 313 keys added.


In [22]:
def retrieve_context_parent_retriever(query: str) -> dict:
    """
    Retrieves relevant PARENT document chunks using LangChain's
    ParentDocumentRetriever, searching via child chunks.
    """
    print(f"\n--- ParentDocumentRetriever Function called with query: '{query}' ---")
    global retriever # Access the globally initialized retriever

    if retriever is None:
        print("  [Function Error] ParentDocumentRetriever is not initialized.")
        return {"status": "error", "message": "Retriever not ready.", "context": ""}

    try:
        # Use the retriever's get_relevant_documents method
        print(f"  Performing retrieval via ParentDocumentRetriever...")
        # This searches child vectors but returns corresponding parent documents
        # Adjust 'k' inside the retriever definition if needed, default search args used here
        retrieved_docs = retriever.get_relevant_documents(query) # Returns List[Document]

        if not retrieved_docs:
            print("  [Function Info] No relevant parent documents found.")
            return {"status": "success", "message": "No relevant context found.", "context": ""}
        else:
            # Format the retrieved PARENT documents' content into a single string
            context_list = []
            print(f"  [Function Info] Retrieved {len(retrieved_docs)} parent document(s).")
            for i, doc in enumerate(retrieved_docs):
                # Include metadata for better context understanding by the LLM and for tracing
                context_chunk = (
                    f"--- Retrieved Context Chunk {i+1} ---\n"
                    f"Source Post ID: {doc.metadata.get('post_id', 'N/A')}\n"
                    # f"Source File: {doc.metadata.get('source', 'N/A')}\n" # Uncomment if loading from multiple files
                    # f"Source URL: {doc.metadata.get('url', 'N/A')}\n" # Optional
                    f"Content: {doc.page_content}"
                )
                context_list.append(context_chunk)
            context_string = "\n\n".join(context_list)

            print(f"  [Function Success] Formatted context from {len(retrieved_docs)} parent document(s).")
            # print(f"DEBUG Context String:\n{context_string[:500]}...") # Uncomment for debugging
            return {"status": "success", "message": "Context retrieved.", "context": context_string}

    except Exception as e:
        print(f"  [Function Error] Error during ParentDocumentRetriever retrieval: {e}")
        traceback.print_exc()
        return {"status": "error", "message": f"An error occurred during retrieval: {str(e)}", "context": ""}

# --- Prepare the tool list for the ADK Agent ---
# The agent's 'tools' argument expects a list of callable functions
langchain_tool_list = []
# Check if the 'retriever' object was successfully created in Cell 5
if 'retriever' in globals() and retriever is not None:
    if callable(retrieve_context_parent_retriever):
        langchain_tool_list = [retrieve_context_parent_retriever]
        print("\nParentDocumentRetriever function prepared for ADK Agent.")
    else:
        # This case should ideally not happen if the function definition is correct
        print("\nError: ParentDocumentRetriever function definition issue.")
else:
    print("\nWarning: ParentDocumentRetriever not ready, tool not prepared (check Cell 5 execution).")


ParentDocumentRetriever function prepared for ADK Agent.


In [23]:
# Define the model name for the ADK Agent
ADK_AGENT_MODEL = 'gemini-2.0-flash' # Or 'gemini-1.5-flash-latest', 'gemini-1.5-pro-latest' etc.

# Agent instructions - referencing the correct function name
rag_agent_instruction = """You are a factual Q&A assistant answering questions about specific Reddit posts. Your task is to answer user questions based *only* on the context provided by the 'retrieve_context_parent_retriever' function. Follow these steps precisely:
1.  When you receive a user query, immediately call the 'retrieve_context_parent_retriever' function with the exact user query to find relevant information from the indexed Reddit posts. This function returns larger chunks of context.
2.  Examine the result from the function, specifically the 'context' field in the returned dictionary. This field contains one or more chunks of text identified by '--- Retrieved Context Chunk X ---' and including metadata like 'Source Post ID'.
3.  If the function returns context (status is 'success' and 'context' is not empty), synthesize an answer based *solely* on the information present in ALL the provided context chunks. Start your response with: "Based on the retrieved context: ". Provide a detailed answer using the information from the context. Do not add any information not present in the retrieved context. You can reference Post IDs if helpful.
4.  If the function returns a message indicating no relevant context was found (e.g., the 'context' field is empty or status is not 'success'), respond *exactly* with: "I could not find relevant information about that in the indexed documents."
5.  Do not use your general knowledge or any information outside the explicitly provided context from the function. If the function returns an error status, state that you encountered an issue retrieving information.
"""

rag_agent = None # Initialize to None
# Check if API key is set AND the LangChain tool list is populated
if os.environ.get('GOOGLE_API_KEY') and langchain_tool_list:
    try:
        rag_agent = Agent(
            name="pdr_rag_agent_v1", # Parent Document Retriever agent
            model=ADK_AGENT_MODEL,
            description="Answers questions using RAG based on Reddit posts, using LangChain's ParentDocumentRetriever via an ADK tool.",
            instruction=rag_agent_instruction,
            tools=langchain_tool_list, # Pass the list containing the PDR retrieval function
        )
        print(f"ADK Agent '{rag_agent.name}' created successfully with model '{ADK_AGENT_MODEL}'.")
        if rag_agent.tools:
             # getattr(tool, '__name__', str(tool)) handles potential non-function items if list was wrong
             print(f"Agent Tools: {[getattr(tool, '__name__', str(tool)) for tool in rag_agent.tools]}")
        else:
             print("Agent Tools: None")
    except Exception as e:
        print(f"ERROR creating ADK Agent (Model: {ADK_AGENT_MODEL}): {e}")
        traceback.print_exc()
else:
    if not os.environ.get('GOOGLE_API_KEY'):
        print("Agent creation skipped: Google API Key is not configured.")
    elif not langchain_tool_list:
        print("Agent creation skipped: LangChain retrieval tool is not ready (check Cell 6).")

ADK Agent 'pdr_rag_agent_v1' created successfully with model 'gemini-2.0-flash'.
Agent Tools: ['retrieve_context_parent_retriever']


In [24]:
adk_runner = None
adk_session_service = None
app_name_for_adk = "PDR_LangChain_ADK_App" # Define specific app name

if rag_agent:
    try:
        adk_session_service = InMemorySessionService()
        adk_runner = Runner(
            agent=rag_agent,
            app_name=app_name_for_adk, # Use defined app name
            session_service=adk_session_service,
        )
        print(f"ADK Runner and Session Service initialized for App: '{app_name_for_adk}'.")
    except Exception as e:
        print(f"Error initializing ADK Runner/SessionService: {e}")
        traceback.print_exc()
        adk_runner = None
        adk_session_service = None
else:
    print("ADK Runner initialization skipped because the Agent was not created.")

ADK Runner and Session Service initialized for App: 'PDR_LangChain_ADK_App'.


In [27]:
# Modified function to handle a single turn within an existing session
async def handle_chat_turn(session_id: str, user_query: str):
    """
    Sends a user query to the agent within a specific session and returns the response.
    Assumes the session already exists.
    """
    if not adk_runner or not adk_session_service:
        print("ERROR: ADK Runner or Session Service is not initialized.")
        return "Error: ADK components not ready."
    if not rag_agent:
        print("ERROR: ADK Agent is not initialized.")
        return "Error: Agent not ready."

    app_name = getattr(adk_runner, 'app_name', app_name_for_adk) # Get app name
    user_id = "colab_chat_user" # Consistent user ID for the chat

    print(f"  Sending to Agent (Session: {session_id}): '{user_query}'")
    final_response = "Agent did not provide a final text response."

    try:
        # Verify session exists before running (optional, but good check)
        try:
            adk_session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_id)
        except Exception as get_e:
            print(f"  ERROR: Cannot get session {session_id} before running turn: {get_e}")
            return f"Error: Session {session_id} may not exist or became invalid."

        # Format the user input
        content = adk_types.Content(role='user', parts=[adk_types.Part(text=user_query)])

        # Run the agent for this turn using the existing session_id
        async for event in adk_runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
            event_type = type(event).__name__
            # Optional: Reduce logging verbosity for chat interface
            # print(f"    [ADK Event]: {event_type}")

            if hasattr(event, 'tool_call') and event.tool_call:
                 tool_name = getattr(event.tool_call, 'name', 'N/A')
                 print(f"      DEBUG: Agent calling tool: '{tool_name}'") # Keep tool call log

            if hasattr(event, 'content') and event.content:
                content_role = getattr(event.content, 'role', 'N/A')
                if hasattr(event.content, 'parts'):
                    for part in event.content.parts:
                        if hasattr(part, 'function_response') and part.function_response:
                             # Optional: Log tool response success/failure
                             fr_name = getattr(part.function_response, 'name', 'N/A')
                             print(f"      DEBUG: Got response from tool '{fr_name}'.")
                        if hasattr(part, 'text') and part.text:
                             if content_role == 'model':
                                 # Capture the final text response from the model for this turn
                                 final_response = part.text
                                 # No need to print partial model text here, we'll print final below

        # print(f"\n--- Agent turn finished for Session: {session_id} ---") # Reduce noise

    except Exception as e:
        print(f"ERROR during chat turn processing for session {session_id}: {e}")
        traceback.print_exc()
        final_response = f"An error occurred: {str(e)}"
    # NOTE: No session deletion here - the session persists for the chat

    return final_response

In [28]:
async def run_interactive_chat():
    """Runs a loop to chat with the RAG agent using a persistent session."""
    if not adk_runner or not adk_session_service:
        print("ERROR: ADK components not ready. Cannot start chat.")
        return
    if not rag_agent:
        print("ERROR: ADK Agent not ready. Cannot start chat.")
        return

    # --- Chat Setup ---
    session_id = f"persistent_chat_{os.urandom(6).hex()}" # Single session ID for the whole chat
    user_id = "colab_chat_user"
    app_name = getattr(adk_runner, 'app_name', app_name_for_adk)
    print("\n--- Starting Interactive Chat ---")
    print(f"Using Session ID: {session_id}")
    print('Type "quit" or "exit" to end the chat.')

    # --- Create the persistent session ONCE ---
    try:
        print(f"Creating chat session {session_id}...")
        adk_session_service.create_session(
            app_name=app_name,
            user_id=user_id,
            session_id=session_id
        )
        print("Session created.")
    except Exception as create_e:
        print(f"FATAL ERROR: Could not create persistent session {session_id}: {create_e}")
        return # Cannot proceed without a session

    # --- Chat Loop ---
    while True:
        try:
            user_input = input("\nYou: ")
            if user_input.lower() in ["quit", "exit"]:
                print("Ending chat.")
                break
            if not user_input.strip():
                continue

            # Call the handler for this turn using the persistent session ID
            agent_response = await handle_chat_turn(session_id, user_input)

            print(f"\nAgent: {agent_response}")

        except KeyboardInterrupt:
            print("\nEnding chat due to keyboard interrupt.")
            break
        except Exception as loop_e:
            print(f"\nAn error occurred in the chat loop: {loop_e}")
            traceback.print_exc()
            # Decide whether to break or continue after an error
            break

    # --- Chat Cleanup (Optional) ---
    try:
        print(f"\nAttempting to delete chat session {session_id}...")
        adk_session_service.delete_session(
            app_name=app_name,
            user_id=user_id,
            session_id=session_id
        )
        print("Chat session deleted.")
    except Exception as del_e:
        print(f"Info/Warning during chat session deletion: {del_e}")

# --- Run the Interactive Chat in Colab ---
# Use await to run the async chat function directly in the notebook cell
await run_interactive_chat()


--- Starting Interactive Chat ---
Using Session ID: persistent_chat_52b37a587cb4
Type "quit" or "exit" to end the chat.
Creating chat session persistent_chat_52b37a587cb4...
Session created.

You: What are the main themes in the documents?
  Sending to Agent (Session: persistent_chat_52b37a587cb4): 'What are the main themes in the documents?'



--- ParentDocumentRetriever Function called with query: 'What are the main themes in the documents?' ---
  Performing retrieval via ParentDocumentRetriever...
  [Function Info] Retrieved 3 parent document(s).
  [Function Success] Formatted context from 3 parent document(s).
      DEBUG: Got response from tool 'retrieve_context_parent_retriever'.

Agent: Based on the retrieved context: The main themes in the documents revolve around trade war implications, particularly between the United States and China (Post ID: 1k4i5n9, 1k76tug), and potential investment opportunities arising from these tensions (Post ID: 1k4i5n9). Tariffs and their impact on specific sectors and companies are also a recurring theme, with companies like MP Materials being highlighted for their potential due to Western alignment and companies like Skechers facing challenges due to reliance on Asian manufacturing and reciprocal tariffs from China (Post ID: 1k4i5n9, 1k76tug). Another theme involves identifying potentia


--- ParentDocumentRetriever Function called with query: 'What are some potential stock options that could yield good returns?' ---
  Performing retrieval via ParentDocumentRetriever...
  [Function Info] Retrieved 4 parent document(s).
  [Function Success] Formatted context from 4 parent document(s).
      DEBUG: Got response from tool 'retrieve_context_parent_retriever'.

Agent: Based on the retrieved context:

*   **BNTX options:** Buying short-dated BNTX options is suggested, anticipating a potential 20%+ stock increase based on results presented on April 27 and 29 (Post ID: 1k7s0er).
*   **MP Materials:** The author holds a mixture of calls and shares in MP Materials, which have already begun to return sizable gains (Post ID: 1k4i5n9).
*   **SPY calls:** Timing calls on SPY in anticipation of news breaking that the US & China have reached a trade deal is mentioned as a potential strategy (Post ID: 1k70nz2). The expectation is a 2%+ jump once the news breaks.
*   **Alphabet (Google)